In [19]:
import sys
import xgboost as xgb
import dask.array as da
import dask.distributed

In [20]:
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [21]:
hosts = ["localhost", "node1", "node2", "node3"]

In [22]:
# cluster = dask.distributed.LocalCluster(n_workers=4, threads_per_worker=1)

In [26]:
cluster = dask.distributed.SSHCluster(
    hosts=hosts,
    worker_options={"nthreads": 2},
    scheduler_options={"port": 0, "dashboard_address": ":8797"},
    remote_python="python3",
)

In [27]:
client = dask.distributed.Client(cluster)

In [28]:
# X and y must be Dask dataframes or arrays
num_obs = 1e5
num_features = 20
X = da.random.random(
    size=(num_obs, num_features)
)
y = da.random.choice(
    a=[0, 1],
    size=num_obs,
    replace=True
)

In [29]:
dtrain = xgb.dask.DaskDMatrix(client, X, y)

In [30]:
output = xgb.dask.train(
    client,
    {'verbosity': 2,
     'tree_method': 'hist',
     'objective': 'binary:logistic'
     },
    dtrain,
    num_boost_round=4, evals=[(dtrain, 'train')])

/usr/local/lib/python3.6/dist-packages/distributed/client.py:3530: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


In [31]:
prediction = xgb.dask.predict(client, output, dtrain)

In [32]:
output

{'booster': <xgboost.core.Booster at 0x7fb530d84b00>,
 'history': {'train': {'error': [0.48706, 0.47274, 0.46635, 0.45928]}}}

In [33]:
prediction

dask.array<from-value, shape=(100000,), dtype=float32, chunksize=(100000,), chunktype=numpy.ndarray>